# Tutorial — Manipulação de dados com **pandas** (versão Python do tutorial em R)

Um dos aspectos mais incríveis do ecossistema Python para ciência de dados é a quantidade de bibliotecas criadas pela comunidade. Assim como no R existem diferentes “gramáticas”, em Python temos estilos e padrões consolidados — aqui vamos focar no **pandas**, que é o “dialeto” mais popular para trabalhar com data frames (tabelas).

Vamos recontar o seu tutorial — mantendo a linguagem e o espírito — mas agora em **Python**.


---
## Instalando e carregando pacotes no Python

No terminal (apenas na primeira vez):

```bash
pip install pandas pyarrow requests
```
No código Python:


In [ ]:
import pandas as pd
import requests  # opcional
pd.__version__

---
## Introdução ao `pandas`

### Começando pelo meio: DataFrames


In [ ]:
df = pd.DataFrame({
    "Candidato": ["Beatriz", "Danilo", "Pedro", "Davi", "Mateus"],
    "Partido":   ["PMDB", "SOL", "PTB", "PSD", "PV"],
    "Votos":     [350, 1598, 784, 580, 2]
})
df

---
## Cadastro de Escolas (SP)

**URL dos dados (CSV):**
```
https://dados.prefeitura.sp.gov.br/dataset/8da55b0e-b385-4b54-9296-d0000014ddd5/resource/533188c6-1949-4976-ac4e-acd313415cd1/download/escolas122024.csv
```

### Abrindo dados em Python (com script)


In [ ]:
url_escolas = "https://dados.prefeitura.sp.gov.br/dataset/8da55b0e-b385-4b54-9296-d0000014ddd5/resource/533188c6-1949-4976-ac4e-acd313415cd1/download/escolas122024.csv"

# sep=';' para equivaler ao read_csv2 do R
escolas = pd.read_csv(url_escolas, sep=";", dtype=str, encoding="utf-8")
escolas.head(6)

### Explorando sem “ver planilha”


In [ ]:
# número de linhas e colunas
escolas.shape

In [ ]:
# nomes das colunas
list(escolas.columns)[:20]

In [ ]:
# estrutura dos dados
escolas.info()

---
## Renomeando variáveis


In [ ]:
escolas = escolas.rename(columns={
    "DRE": "dre_abreviatura",
    "CODESC": "codigo",
    "TIPOESC": "tipo",
    "NOMES": "nome",
    "DIRETORIA": "dre",
    "LATITUDE": "lat",
    "LONGITUDE": "lon",
    "CODINEP": "codigo_inep"
})
list(escolas.columns)[:12]

---
## Uma “gramática”, duas formas (encadeamento de métodos à la pipe)


In [ ]:
# Recarrega e encadeia (method chaining) para ficar auto-contido
escolas = (
    pd.read_csv(url_escolas, sep=";", dtype=str, encoding="utf-8")
      .rename(columns={
          "DRE": "dre_abreviatura",
          "CODESC": "codigo",
          "TIPOESC": "tipo",
          "NOMES": "nome",
          "DIRETORIA": "dre",
          "LATITUDE": "lat",
          "LONGITUDE": "lon",
          "CODINEP": "codigo_inep"
      })
      # seleciona colunas de interesse
      .loc[:, ["dre_abreviatura", "codigo", "tipo", "nome", "dre", "lat", "lon", "codigo_inep"]]
)
escolas.head()

---
## Transformando variáveis (`mutate` → `.assign` / operações diretas)

Vamos ajustar latitude/longitude para formato numérico com separador decimal correto.


In [ ]:
# Converter lat/lon
escolas = (
    escolas
    .assign(
        lat=lambda d: pd.to_numeric(d["lat"], errors="coerce") / 1_000_000,
        lon=lambda d: pd.to_numeric(d["lon"].str.replace(".", "", regex=False), errors="coerce") / 1_000_000
    )
)
escolas[["lat","lon"]].head()

---
## Filtrando linhas (`filter` → máscaras booleanas)

Ex.: selecionar apenas **EMEIs**.


In [ ]:
emeis = escolas[escolas["tipo"] == "EMEI"].copy()
emeis.head()

Ex.: **CEI DIRET**, **CEI INDIR** e **CR.P.CONV** (equivalente ao `OR`).

In [ ]:
tipos_creche = ["CEI DIRET", "CEI INDIR", "CR.P.CONV"]
creches = escolas[escolas["tipo"].isin(tipos_creche)].copy()
creches.head()

E combinando com outra variável (equivalente ao `AND`).

In [ ]:
creches_ipiranga = creches[creches["dre"] == "DIRETORIA REGIONAL DE EDUCACAO IPIRANGA"].copy()
creches_ipiranga.head(), creches_ipiranga.shape

---
## Extras úteis (contagens, ordenações, agrupamentos)


In [ ]:
# Amostra aleatória
escolas.sample(6, random_state=42)

In [ ]:
# Contagem de categorias
escolas["tipo"].value_counts(dropna=False).head(10)

In [ ]:
# Agrupar e sumarizar (groupby + agg)
(
    escolas
    .groupby("dre", dropna=False)
    .agg(qtd=("codigo", "count"))
    .reset_index()
    .sort_values("qtd", ascending=False)
    .head(10)
)

---
## Pipeline completo (do zero ao subset final)


In [ ]:
url_escolas = "https://dados.prefeitura.sp.gov.br/dataset/8da55b0e-b385-4b54-9296-d0000014ddd5/resource/533188c6-1949-4976-ac4e-acd313415cd1/download/escolas122024.csv"
tipos_creche = ["CEI DIRET", "CEI INDIR", "CR.P.CONV"]

escolas_pipeline = (
    pd.read_csv(url_escolas, sep=";", dtype=str, encoding="utf-8")
      .rename(columns={
          "DRE": "dre_abreviatura",
          "CODESC": "codigo",
          "TIPOESC": "tipo",
          "NOMES": "nome",
          "DIRETORIA": "dre",
          "LATITUDE": "lat",
          "LONGITUDE": "lon",
          "CODINEP": "codigo_inep"
      })
      .assign(
          lat=lambda d: pd.to_numeric(d["lat"], errors="coerce") / 1_000_000,
          lon=lambda d: pd.to_numeric(d["lon"].str.replace(".", "", regex=False), errors="coerce") / 1_000_000
      )
      .loc[:, ["dre_abreviatura", "codigo", "tipo", "nome", "dre", "lat", "lon", "codigo_inep"]]
)

creches_ipiranga = (
    escolas_pipeline[
        escolas_pipeline["tipo"].isin(tipos_creche) &
        (escolas_pipeline["dre"] == "DIRETORIA REGIONAL DE EDUCACAO IPIRANGA")
    ]
    .copy()
)

escolas_pipeline.head(), creches_ipiranga.head(), creches_ipiranga.shape